In [52]:
import pandas as pd
import numpy as np
import os

In [53]:
dataset = pd.read_csv(os.path.join(os.getcwd(),'dataset/train.csv'))
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
dataset.describe()
dataset.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [55]:
independent = dataset.drop('label',axis=1)
independent.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [56]:
y = dataset['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [57]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [33]:
vocabSize = 10000
messages = independent.copy()
messages.reset_index(inplace=True)
# messages = independent.copy().reset_index(inplace=True)

In [34]:
import nltk,re
from nltk.corpus import stopwords as stw
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Vedang
[nltk_data]     Parasnis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
messages['title']
len(messages)

20800

In [60]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stw.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [61]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [64]:
onehotRep = [one_hot(words,vocabSize) for words in corpus]
onehotRep

[[3382, 7292, 9258, 5960, 2036, 3479, 4632, 8237, 1619, 6653],
 [3486, 4857, 1138, 3450, 7000, 312, 7354],
 [7472, 5246, 8912, 1672],
 [5858, 5864, 7265, 476, 7367, 5755],
 [7308, 7000, 5, 7526, 411, 5223, 7000, 208, 8772, 372],
 [8952,
  7850,
  9798,
  7281,
  5167,
  8074,
  3749,
  6184,
  1888,
  4027,
  8548,
  2541,
  6341,
  2174,
  7354],
 [7755, 7755, 2664, 7184, 3002, 3169, 1705, 7869, 970, 9121, 6948, 1015],
 [4693, 5089, 171, 7928, 8813, 7715, 5310, 7895, 8117, 9236, 2768],
 [3643, 4073, 5010, 2116, 8074, 9951, 1534, 809, 566, 5418, 8117, 9236, 2768],
 [3835, 3510, 3387, 4103, 4413, 2667, 8074, 6403, 8117, 9236, 2768],
 [8980, 4466, 539, 9245, 244, 4157, 8007, 1630, 8074, 4286],
 [4929, 4140, 4415, 758, 5664, 9000, 6094, 8217],
 [2476, 4779, 4707, 2561, 4968, 7206, 1168, 8923, 5984, 6963, 7172],
 [476, 169, 2036, 4157, 8074, 4413],
 [9397, 9705, 7363, 3087, 6293, 317, 2709, 8249, 5376],
 [6323, 8580, 7067, 4721, 7894, 8576, 9022, 8117, 9236, 2768],
 [5088, 2076, 1543, 9736

In [65]:
### add padding to embedded sequence 
sentence_length = 30
embeddedStartLayer = pad_sequences(onehotRep,padding='pre',maxlen=sentence_length)
print(embeddedStartLayer)

[[   0    0    0 ... 8237 1619 6653]
 [   0    0    0 ... 7000  312 7354]
 [   0    0    0 ... 5246 8912 1672]
 ...
 [   0    0    0 ... 8117 9236 2768]
 [   0    0    0 ... 9070 5283 3648]
 [   0    0    0 ... 1636 4320 3835]]


In [68]:
embedding_vectors = 50
model = Sequential()
model.add(Embedding(vocabSize,embedding_vectors,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            500000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 560,501
Trainable params: 560,501
Non-trainable params: 0
_________________________________________________________________


In [77]:
embedding_vectors = 50
blstm = Sequential()
blstm.add(Embedding(vocabSize,embedding_vectors,input_length=sentence_length))
blstm.add(Bidirectional(LSTM(100)))
blstm.add(Dense(1,activation='sigmoid'))
blstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
blstm.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 50)            500000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               120800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 621,001
Trainable params: 621,001
Non-trainable params: 0
_________________________________________________________________


In [81]:
from sklearn.model_selection import train_test_split
X_input = np.array(embeddedStartLayer)
y_input = np.array(y)
X_input.shape

(20800, 30)

In [82]:
y_input.shape

(20800,)

In [85]:
X_train,X_test,Y_train,Y_test = train_test_split(X_input,y_input,test_size=0.4,random_state=42)

In [87]:
blstm.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=65)

Train on 12480 samples, validate on 8320 samples
Epoch 1/10
12480/12480 [==============================] - 23s 2ms/sample - loss: 0.3371 - acc: 0.8548 - val_loss: 0.1840 - val_acc: 0.9270
Epoch 2/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.1281 - acc: 0.9514 - val_loss: 0.1878 - val_acc: 0.9286
Epoch 3/10
12480/12480 [==============================] - 19s 2ms/sample - loss: 0.0787 - acc: 0.9727 - val_loss: 0.1989 - val_acc: 0.9236
Epoch 4/10
12480/12480 [==============================] - 21s 2ms/sample - loss: 0.0501 - acc: 0.9834 - val_loss: 0.2239 - val_acc: 0.9261
Epoch 5/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0280 - acc: 0.9921 - val_loss: 0.2673 - val_acc: 0.9154
Epoch 6/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0168 - acc: 0.9958 - val_loss: 0.3282 - val_acc: 0.9215
Epoch 7/10
12480/12480 [==============================] - 21s 2ms/sample - loss: 0.0059 - acc: 0.9986 - val_loss: 0.4

In [88]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=65)

Train on 12480 samples, validate on 8320 samples
Epoch 1/10
12480/12480 [==============================] - 22s 2ms/sample - loss: 0.3245 - acc: 0.8612 - val_loss: 0.1919 - val_acc: 0.9256
Epoch 2/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.1257 - acc: 0.9526 - val_loss: 0.1774 - val_acc: 0.9298
Epoch 3/10
12480/12480 [==============================] - 19s 2ms/sample - loss: 0.0666 - acc: 0.9775 - val_loss: 0.2026 - val_acc: 0.9308
Epoch 4/10
12480/12480 [==============================] - 19s 2ms/sample - loss: 0.0338 - acc: 0.9901 - val_loss: 0.2321 - val_acc: 0.9238
Epoch 5/10
12480/12480 [==============================] - 19s 2ms/sample - loss: 0.0148 - acc: 0.9954 - val_loss: 0.3068 - val_acc: 0.9260
Epoch 6/10
12480/12480 [==============================] - 19s 2ms/sample - loss: 0.0076 - acc: 0.9974 - val_loss: 0.3699 - val_acc: 0.9233s: 0.0076 -
Epoch 7/10
12480/12480 [==============================] - 19s 2ms/sample - loss: 0.0045 - acc: 0.9987 - va